In [ ]:
# code to generate yearly summaries of DUNE data volumes from input parameters
# rewritten from the version in the CDR - mainly by using maps of years instead of arrays to make it clearer what is in each year.
# HMS 2022-10-23


import os,sys,string,time,commentjson,datetime, math
from csv import reader
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

DEBUG = False
DRAW = True
import numpy as np

from NumberUtils import dump
from NumberUtils import DrawTex
from NumberUtils import cumulateMap
from NumberUtils import DrawDet
from NumberUtils import DrawType
from NumberUtils import makeArray
from NumberUtils import ToCSV1
from NumberUtils import ToCSV2
from NumberUtils import SumOver1
from NumberUtils import SumOver2
from NumberUtils import TableTex
from NumberUtils import DrawTex

In [ ]:
# read in a configfile
configfilename = "Parameters_2022-11-07-2040.json"

#if len(sys.argv) > 1:
#  configfile = sys.argv[1]

shortname = "report/"+configfilename.replace(".json","")
if os.path.exists(configfilename):
  with open(configfilename,'r') as f:
    config = commentjson.load(f)
else:
  print ("no config file",configfilename)
  sys.exit(0)

if not "Version" in config or config["Version"] < 5:
  print (" this code expects Version >= 2")
  sys.exit(1)

json_formatted_str = commentjson.dumps(config, indent=2)

In [ ]:
MaxYear = config["MaxYear"]
config["filename"] = configfilename.replace("_","\_")
MinYear = config["MinYear"]
Detectors = config["Detectors"]
if DEBUG:
  Detectors = ["SP","SP2","DP"]
Years = np.array(config["Years"])
#if DEBUG:
#  Years = Years[0:7]
shortname = shortname.replace("2040","%d"%MaxYear)
# make a tex output file
texfilename = shortname.replace("report/","")+".tex"
texfile = open(texfilename,'w')
tablefile = open("report/tables.tex",'w')
template = open("Header.tex",'r')
header = template.readlines()
for l in header:
    texfile.write(l)
print (Years, len(Years))
size = len(Years)

Units = config["Units"]
Formats = config["Formats"]

Detectors = config["Detectors"]

Cap = config["Cap"]

BaseMemory = config["Base-Memory"]

print (Detectors)

CombinedDetectors = config["CombinedDetectors"]

DetectorParameters = list(config["SP"].keys())

if "Comment" in DetectorParameters:
    DetectorParameters.remove("Comment")

TapeLifetimes = config["TapeLifetimes"]

DiskLifetimes = config["DiskLifetimes"]

TapeCopies = config["TapeCopies"]

DiskCopies = config["DiskCopies"]

# this is how far you go back each time you reprocess reco.
Reprocess = config["Reprocess"]

PerYear = config["PerYear"]

StorageTypes = list(TapeCopies.keys())

# plot config
DetColors=config["DetColors"]
DetLines = config["DetLines"]
TypeColors=config["TypeColors"]
TypeLines = config["TypeLines"]

PatternFraction = config["PatternFraction"]

SplitsYear = config["SplitsYear"]
SplitsEarly = config["SplitsEarly"]
SplitsLater = config["SplitsLater"]

Explain = config["Explain"]
Explain["filename"] = "Input configuration file"

In [ ]:
for f in Explain.keys():
    
    texfile.write("{\\tt %s:} %s = {\\tt %s} \\\\\n"%(f,Explain[f], config[f]))

In [ ]:
dofirst = ["Events","Test","Sim-Events"]
print ("Detector Parameters",DetectorParameters)
# read in the raw information

Inputs = {}
for det in Detectors:
  Inputs[det]={}
  for type in dofirst:
      Inputs[det][type]={}
      for year in Years:
          Inputs[det][type][year] = float(config[det][type][year-Years[0]])

# fill in other useful arrays
for det in Detectors:
  if DEBUG: print ("Events",det,Inputs[det]["Events"])
  #print ("see it", det,Inputs[det].keys())
  for key in DetectorParameters:
    #print(key,det)
    # skip the ones already done
    if key in dofirst:
      continue
    
    # sim has its own configuration
    # print ("this is the key",det,key)
    
    if key in ["Reco-CPU","Reco"]:  # if doing reco, do over previous events using memory
        Inputs[det][key] = cumulateMap(Years,Inputs[det]["Events"],Reprocess[det])
        for year in Years:
            Inputs[det][key][year] *= config[det][key]
        if key == "Reco-CPU":
            for year in Years:
                Inputs[det][key][year] *= (config[det]["Reco-Memory"]/BaseMemory)
        continue
        
    if key == "Raw":
        Inputs[det][key] ={}
        for year in Years:
            Inputs[det][key][year] = Inputs[det]["Events"][year]*config[det][key]
        continue
        
    if key in ["Sim","Sim-CPU"]:
        
        Inputs[det][key] ={}
        for year in Years:
            Inputs[det][key][year]=Inputs[det]["Sim-Events"][year]*config[det][key]
            if key == "Sim-CPU":
                Inputs[det][key][year]*=(config[det]["Sim-Memory"]/BaseMemory)
        continue
    if DEBUG: print ("other key",det,key)

for det in Inputs.keys():
    #print ("det",det,Inputs[det].keys())
    
    if "Sim-Memory" in Inputs[det]:
        Inputs[det].pop("Sim-Memory")
    if "Reco-Memory" in Inputs[det]:
        Inputs[det].pop("Reco-Memory")

In [ ]:
# make a data file which uses # of events to figure out how big samples are

if PerYear["Reco"]!=PerYear["Reco-CPU"]:
    print ("Data growth has to match reprocessing cycles/year")
    PerYear["Reco"] = PerYear["Reco-CPU"]
if PerYear["Sim"]!=PerYear["Sim-CPU"]:
    print ("Sim growth has to match reprocessing cycles/year")
    PerYear["Sim"] = PerYear["Sim-CPU"]

Data = {}
dump = open("dump.txt",'w')

    
#print (Inputs.keys())
fields = list(Inputs["ND"].keys())
#print ("fields",fields)
for dtype in fields:
  Data[dtype] = {}
  if "Memory" in dtype:
        continue
  for det in Inputs.keys():
    Data[dtype][det] = {}
    # this allows you to, say, do 2 passes of reco/year
    for year in Years:
        Data[dtype][det][year] = Inputs[det][dtype][year] * float(PerYear[dtype])
    # compensate for nominal units being millions and TB or singles and MB
    if Units[dtype] == "PB":
        for year in Years:
            Data[dtype][det][year] *= 0.001
    ds = "data %s %s %f\n"%(dtype,det,Data[dtype][det][2022])
    dump.write(ds)

In [ ]:
# dtype = "Raw+Test"
# Data[dtype] = {}
# for det in Inputs.keys():
#     Data[dtype][det] = {}
    
#     for year in Years:
#         Data[dtype][det][year] =  Data["Raw"][det][year]+Data["Test"][det][year]
#     # compensate for nominal units being millions and TB or singles and MB



In [ ]:
# impose a cap at Cap on things derived from raw data

dtype = "Raw"

Data["Raw"]["Total"] = {}
for year in Years:
        Data[dtype]["Total"][year] = 0.0
for det in Inputs.keys():
    for year in Years:
        Data[dtype]["Total"][year] +=  Data["Raw"][det][year]
        
dtypes = ["Raw","Reco-CPU"]
for dtype in dtypes:
    for det in Inputs.keys():
        #print (dtype,det,2035,1.0,Data[dtype][det][2035] )
        for year in Years:
            cap = Data["Raw"]["Total"][year]/Cap
           # print (dtype,det,year,cap,Data[dtype][det][year] )
            if cap > 1:
                Data[dtype][det][year] /=cap
        #print (dtype,det,2035,cap,Data[dtype][det][2035] )

# build analysis

Data["Analysis-CPU"]= {}

for det in Inputs.keys():
    Data["Analysis-CPU"][det] = {}
    for year in Years:
        Data["Analysis-CPU"][det][year] = (Data["Reco-CPU"][det][year]+Data["Sim-CPU"][det][year])*config["Analysis"]["Scale"]  
 
# then make the sums


dtypes = ["Raw","Reco-CPU","Sim-CPU","Analysis-CPU"]
for dtype in dtypes:
    Data[dtype]["Total"] ={}
    for year in Years:
        Data[dtype]["Total"][year] = 0.0
    for det in Inputs.keys():
        if dtype != "Analysis":
            for year in Years:
               Data[dtype]["Total"][year]+=  Data[dtype][det][year] 


             
    

In [ ]:
PlotYears = []
for i in range(MinYear,MaxYear+1):
    PlotYears.append(i)
#PlotYears = Years
print ("PlotYears",PlotYears)
# draw things
things = list(Inputs.keys())+["Total"]

if DRAW:
    for stuff in ["Events","Sim-Events","Raw","Reco","Sim","Reco-CPU","Sim-CPU"]:
        DrawDet(shortname,stuff,PlotYears,Data,things,Units,DetColors,DetLines)
    

# if DRAW:
#     DrawDet(shortname,"Events",PlotYears,Data,Inputs.keys(),Units,DetColors,DetLines)
#     DrawDet(shortname,"Reco-CPU",PlotYears,Data,Inputs.keys(),Units,DetColors,DetLines)
#     DrawDet(shortname,"Sim-CPU",PlotYears,Data,Inputs.keys(),Units,DetColors,DetLines)
#     DrawDet(shortname,"Raw",PlotYears,Data,Inputs.keys(),Units,DetColors,DetLines)
#     DrawDet(shortname,"Test",PlotYears,Data,Inputs.keys(),Units,DetColors,DetLines)
#     DrawDet(shortname,"Sim Events",PlotYears,Data,Inputs.keys(),Units,DetColors,DetLines)
#     DrawDet(shortname,"Sim",PlotYears,Data,Inputs.keys(),Units,DetColors,DetLines)
#     DrawDet(shortname,"Reco-CPU",PlotYears,Data,list(Inputs.keys())+["Total"],Units,DetColors,DetLines)
#     DrawDet(shortname,"Sim-CPU",PlotYears,Data,list(Inputs.keys())+["Total"],Units,DetColors,DetLines)
#     DrawDet(shortname,"Raw",PlotYears,Data,list(Inputs.keys())+["Total"],Units,DetColors,DetLines)
  
# for t in Data.keys():
#     print (t)
    
# Captions1 = {"Events":"Million of detector events per year projected",
# "Test":"PB of Test data projected",
# "Sim-Events":"Millions of simulated events per year projected",
# "Raw":"Raw data written per year in PB",
# "Reco-CPU":"CPU needs in core-years for data reconstruction. \
#              Slot weighted wall time takes into account memory use.  Assumes rereconstruction of older data.",
# "Sim-CPU":"CPU needs in core-years for simulation and reconstruction. \
#              Slot weighted wall time takes into account memory use."
# "Reco":
# Sim
# Total-CPU
# Cores
# HS06
# WALL
# Total
# Collab
# FNAL
# CERN

# for key in Data.keys():
#     ToCSV2("Table-"+key,key,PlotYears,Data,Units,Formats)
#     s = TableTex("Table-"+key,key+" values by detector.",key+"\n")ions
#     s2 = DrawTex(shortname,"Total-CPU.png","CPU time in Wall Hours/year. Squares are measured values for 2021.","TotalCPU")

#     print ("tex:",s)
#     texfile.write(s)

In [ ]:
# merge far detector info

print ("Data keys",Data.keys())

for dtype in Data.keys():
    print ("Merge protodunes",dtype)
    det = "ProtoDUNEs" 
    Data[dtype][det] = {}
    for year in Years:  
        Data[dtype][det][year] = Data[dtype]["SP"][year] + Data[dtype]["DP"][year] + Data[dtype]["SP2"][year] + Data[dtype]["PDVD"][year]

    Data[dtype].pop("SP")
    Data[dtype].pop("SP2")
    Data[dtype].pop("DP")
    Data[dtype].pop("PDVD")
    


In [ ]:
# merge far detector into "FDs
for dtype in Data.keys():
    det = "FDs"
    print ("merge FDS",dtype)
    Data[dtype][det] =  {}
    for year in Years:  
        Data[dtype][det][year] = Data[dtype]["HD"][year] + Data[dtype]["VD"][year]
    Data[dtype].pop("HD")
    Data[dtype].pop("VD")


In [ ]:
# make a total CPU category

Data["Total-CPU"]={}

for det in CombinedDetectors:
    Data["Total-CPU"][det] =  {}
    for year in Years:
        Data["Total-CPU"][det][year] = Data["Reco-CPU"][det][year] + Data["Sim-CPU"][det][year]
    #print(det,Data["Total-CPU"][det])

In [ ]:
# make totals across categories. 

DataTypes = list(Data.keys())

for dt in DataTypes:
    Data[dt]["Total"] = {}
    for year in Years:
        Data[dt]["Total"][year]=0.0
    for k in Data[dt].keys():
        if k == "Total":
          continue  
        for year in Years:
            Data[dt]["Total"][year] += Data[dt][k][year]
    

In [ ]:
# and make a special data type for cores

Data["Cores"] = {}
Data["HS06"] = {}
#Data["WALL"] = {}
 
MHrsPerYear = 1000000./365./24.
for k in Data["Total-CPU"].keys():
    if "MARS" not in k :
        efficiency = config["Cores"]["Efficiency"]
    else:
        efficiency = 1

    scaleTo2020 = config["Cores"]["2020Units"]
    Data["Cores"][k]={}
    Data["HS06"][k]={}
#    Data["WALL"][k]={}
    for year in Years:
        Data["Cores"][k][year] = Data["Total-CPU"][k][year]*MHrsPerYear/efficiency/scaleTo2020
        Data["HS06"][k][year] = Data["Total-CPU"][k][year]*MHrsPerYear/efficiency/scaleTo2020*config["kHEPSPEC06PerCPU"]
#        Data["WALL"][k][year] = Data["Total-CPU"][k][year]*MHrsPerYear/efficiency/scaleTo2020


In [ ]:
#  for Storage work out split between different institutions

Splits = {}
for f in SplitsEarly:
    Splits[f] = {}
    for t in SplitsEarly[f]:
        Splits[f][t] = {}
        for loc in SplitsEarly[f][t]: 
            Splits[f][t][loc] = {}
            #print (f,t,Splits[f][t],Splits[f][t][0])
    
            for y in Years:
                if y < SplitsYear:
                    Splits[f][t][loc][y]=SplitsEarly[f][t][loc]
                else:
                    Splits[f][t][loc][y]=SplitsLater[f][t][loc]

In [ ]:
# now do some Cumulative-work.  Stuff stays on tape/disk for different amounts of time and we have multiple copies

Storage = {}
for k in StorageTypes:
    Storage[k] = {}
Storage["Total"] = {}
Storage["Collab"] = {}
Storage["FNAL"] = {}
Storage["CERN"] = {}
Storage["Total"]["Cumulative-Tape"] = {}
Storage["Total"]["Cumulative-Disk"] = {}
Storage["FNAL"]["Cumulative-Tape"] = {}
Storage["FNAL"]["Cumulative-Disk"] = {}
Storage["CERN"]["Cumulative-Tape"] = {}
Storage["CERN"]["Cumulative-Disk"] = {}
Storage["Collab"]["Cumulative-Tape"] = {}
Storage["Collab"]["Cumulative-Disk"] = {}


for year in Years:
    Storage["Total"]["Cumulative-Tape"][year] = 0.0
    Storage["Total"]["Cumulative-Disk"][year] = 0.0

for k in StorageTypes:
    Storage[k]["Tape"] = {}
    Storage[k]["Disk"] = {}
    for year in Years:
        Storage[k]["Tape"][year] = Data[k]["Total"][year]*TapeCopies[k]
        Storage[k]["Disk"][year] = Data[k]["Total"][year]*DiskCopies[k]
    Storage[k]["Cumulative-Tape"] = cumulateMap(Years,Storage[k]["Tape"],TapeLifetimes[k])
    Storage[k]["Cumulative-Disk"] = cumulateMap(Years,Storage[k]["Disk"],DiskLifetimes[k])
    
    for year in Years:
        Storage["Total"]["Cumulative-Tape"][year] += Storage[k]["Cumulative-Tape"][year]
        Storage["Total"]["Cumulative-Disk"][year] += Storage[k]["Cumulative-Disk"][year]
        
for loc in Splits["Disk"]["Raw"]:
    for year in Years:
        Storage[loc]["Cumulative-Disk"][year] = 0.0
        Storage[loc]["Cumulative-Tape"][year] = 0.0       
        for k in StorageTypes:
              Storage[loc]["Cumulative-Disk"][year] += Storage[k]["Cumulative-Disk"][year]*Splits["Disk"][k][loc][year]
              Storage[loc]["Cumulative-Tape"][year] += Storage[k]["Cumulative-Tape"][year]*Splits["Tape"][k][loc][year]


# cdisk = SumOver1("Cumulative-Disk",Data)
# print ("sum over",cdisk)

# for year in Years:
#         Data[loc]["Cumulative-Disk"][year] = 0.0
#         Data[loc]["Cumulative-Tape"][year] = 0.0       
#         for k in StorageTypes:
#               Data[loc]["Cumulative-Disk"][year] += Data[k]["Cumulative-Disk"][year] 
#               Data[loc]["Cumulative-Tape"][year] += Data[k]["Cumulative-Tape"][year] 


In [ ]:

texfile.write("\\section{Disk and Tape needs by source and site}\n")
ToCSV1("Disk_by_location","Cumulative-Disk",PlotYears,Storage,Units,Formats)
ToCSV1("Tape_by_location","Cumulative-Tape",PlotYears,Storage,Units,Formats)
s = "\\begin{table}[h]\n \\centering\\csvautotabularright\
{DUNERSEUSAGE-2022-11-14.csv}\n \\label{Cumulative-Tape}\n\
\\caption{Rucio report on storage usage 2022-11-14 from the Scotgrid Dashboard \
\\href{https://dune.monitoring.edi.scotgrid.ac.uk/app/dashboards}{https://dune.monitoring.edi.scotgrid.ac.uk/app/dashboards}.}\n \\end{table}\n"
s.replace("_","\_")
texfile.write(s)

s = TableTex("Disk_by_location","Disk requests by location. The top 4 lines show the source, the bottom 4 show the locations requested and the total request.","Cumulative-Disk"+"\n")
texfile.write(s)
s = TableTex("Tape_by_location","Tape requests by location. The top 4 lines show the source, the bottom 4 show the locations requested and the total request.","Cumulative-Tape"+"\n")
texfile.write(s)




In [ ]:
# now do some plots

Types = ["ProtoDUNEs","FDs","ND", "Analysis","Total"]

cpuactual = []
coreactual = []
wallactual = []



DrawDet(shortname,"Total-CPU",PlotYears,Data,Types,Units,DetColors,DetLines,cpuactual)
DrawDet(shortname,"Cores",PlotYears,Data,Types,Units,DetColors,DetLines,coreactual)
#DrawDet(shortname,"WALL",PlotYears,Data,Types,Units,DetColors,DetLines,wallactual)
DrawDet(shortname,"HS06",PlotYears,Data,Types,Units,DetColors,DetLines,wallactual)

for x in ["Total-CPU","Cores","HS06"]:
    ToCSV2("report/Table-"+x,x,PlotYears,Data,Units,Formats)

In [ ]:
for t in Data.keys():
    print (t)
    
Captions2 = {"Events":"Million of detector events per year projected",
"Test":"PB of Test data projected",
"Sim-Events":"Millions of simulated events per year projected",
"Raw":"Raw data written per year in PB",
"Reco-CPU":"CPU needs in core-years for data reconstruction. \
             Slot weighted wall time takes into account memory use.  Assumes rereconstruction of older data.",
"Sim-CPU":"CPU needs in core-years for simulation and reconstruction. \
             Slot weighted wall time takes into account memory use.",
"Reco":"PB of reconstructed files/year",
"Sim":"PB of simulated files/year",
"Total-CPU":"Total CPU needs in core-years. Slot weighted wall time takes into account memory use.",
"Cores":"",
"HS06":"",
"WALL":"",
"Analysis-CPU":"Analysis CPU needs in core-years.",
            }
print (Data["Events"]["ProtoDUNEs"])
print (Data["Events"]["FDs"])
print (Data["Events"]["ND"])
for key in Data.keys():
    if not key in Units:
        print ("no units for key",key)
        continue
    ToCSV2("report/Table-"+key,key,PlotYears,Data,Units,Formats)
    s = TableTex("report/Table-"+key,Captions2[key],key+"\n")
    print (key,Data[key].keys())
    DrawDet(shortname,key,PlotYears,Data,list(Data[key].keys()),Units,DetColors,DetLines)
    s2 = DrawTex(shortname,key+".png",Captions2[key],key)
    #texfile.write("\\pagebreak\n")
    texfile.write(s2)
    tablefile.write(s)
    

In [ ]:
print (Storage.keys())
    
Captions1 = {"Cumulative-Tape":"Cumulative Tape needs in PB. Includes data lifetimes", 
             "Cumulative-Disk":"Cumulative Tape needs in PB. Includes data lifetimes"}
            

for key in ['Cumulative-Tape', 'Cumulative-Disk']:
    if not key in Units:
        print ("no units for key",key)
        continue
    ToCSV1("report/Table-"+key,key,PlotYears,Storage,Units,Formats)
    s = TableTex("report/Table-"+key,Captions1[key],key+"\n")
    print (key,s)
    DrawType(shortname,key,PlotYears,Storage,StorageTypes+["Total"],Units,TypeColors,TypeLines)
    s2 = DrawTex(shortname,key+".png",Captions1[key],key)

    texfile.write(s2)
    texfile.write(s)

In [ ]:

tapepoints = np.zeros(len(Years))
diskpoints = np.zeros(len(Years))

#DrawType(shortname,"Tape",Years,Data,StorageTypes+["Total"],Units,TypeColors,TypeLines,None,None)
DrawType(shortname,"Cumulative-Tape",PlotYears,Storage,StorageTypes+["Total"],Units,TypeColors,TypeLines,None,None)
DrawType(shortname,"Cumulative-Disk",PlotYears,Storage,StorageTypes+["Total"],Units,TypeColors,TypeLines,None,None)
#DrawType(shortname,"Cumulative-Disk",Years,Data,StorageTypes+["Total"],Units,TypeColors,TypeLines,None,None)



In [ ]:
tablefile.close()
texfile.write("\\input{tables.tex}\n")
texfile.write("\\end{document}\n")
texfile.close()

In [ ]:
cmd='pdflatex %s'%texfilename
get_ipython().system('{cmd}')